# Types and Dispatch in Julia

Julia is built around types.

Software architectures in Julia are built around good use of the type system.

# Abstract vs concrete types

*Concrete types* are the types of objects. They specify the data structure of an object.

*Abstract types* cannot be instantiated. They define sets of related concrete types (their descendants) by their behavior.

In [1]:
typeof(3)

Int64

In [2]:
typeof(3.0)

Float64

In [3]:
isconcretetype(Float64)

true

In [4]:
isabstracttype(Number)

true

In [5]:
isabstracttype(Real)

true

### Duck typing

A `Number` is some abstract type that can do things like `+`,`-`,`*`, and `/`. In this category we have (concrete) things like `Float64` and `Int32`.

An `AbstractArray` is a type that can be indexed like `A[i]`. An `AbstractArray` may be mutable, meaning it can be set: `A[i]=v`.

### Inspecting the type tree

In [6]:
supertype(Float64)

AbstractFloat

In [7]:
supertype(AbstractFloat)

Real

In [8]:
subtypes(AbstractFloat)

4-element Vector{Any}:
 BigFloat
 Float16
 Float32
 Float64

In [9]:
supertype(Real)

Number

In [10]:
supertype(Number)

Any

Everything is a subtype of `Any`

In [11]:
Number <: Any

true

In [12]:
Float64 <: Any

true

In [13]:
Int32 <: Any

true

In [14]:
Int32 <: String

false

There is also `isa` for objects:

In [15]:
3.0 isa Float64

true

In [16]:
3 isa Float64

false

In [17]:
typeof(3) <: Float64

false

We define a function that, given a concrete type `T`, prints the single branch of the type tree that leads from the top node `Any` to the leave `T`.

In [18]:
show_supertypes(T) = print(join(supertypes(T), " <: "))

show_supertypes (generic function with 1 method)

In [19]:
show_supertypes(Float64)

Float64 <: AbstractFloat <: Real <: Number <: Any

In [20]:
show_supertypes(String)

String <: AbstractString <: Any

Let's extract a branch of the type tree

In [23]:
using AbstractTrees
AbstractTrees.children(x) = subtypes(x)

In [24]:
print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


Note that **concrete types are the leaves of the type tree**.

**Abstract types are nodes** in the type graph.

# Functions, Methods, and Dispatch

Let's define a *function* that calculates the absolute value of a number (like Julias `abs` already does).

How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?

Presumably:
* Real number: "Drop the sign." => `myabs(-4.32) = 4.32`
* Complex number: "Square root of z times the complex conjugate of z." => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [25]:
myabs(x::Float64) = sign(x) * x

myabs (generic function with 1 method)

In [26]:
myabs(-4.32)

4.32

In [27]:
myabs(1.0 + 1.0im)

LoadError: MethodError: no method matching myabs(::ComplexF64)
[0mClosest candidates are:
[0m  myabs([91m::Float64[39m) at In[25]:1

In [28]:
myabsthatdoesntexist(1.0 + 1.0im)

LoadError: UndefVarError: myabsthatdoesntexist not defined

In [29]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

myabs (generic function with 2 methods)

In [30]:
myabs(1.0 + 1.0im)

1.4142135623730951

In [31]:
methods(myabs)

# 2 methods for generic function "myabs":
[1] myabs(x::Float64) in Main at In[25]:1
[2] myabs(z::ComplexF64) in Main at In[29]:1

One can check which particular method is being used through the `@which` macro.

In [32]:
@which myabs(-4.32)

myabs(x::Float64) in Main at In[25]:1

In [33]:
@which myabs(1.0 + 1.0im)

myabs(z::ComplexF64) in Main at In[29]:1

Note that we should better loosen our type restrictions:

In [34]:
myabs(-3)

LoadError: MethodError: no method matching myabs(::Int64)
[0mClosest candidates are:
[0m  myabs([91m::Float64[39m) at In[25]:1
[0m  myabs([91m::ComplexF64[39m) at In[29]:1

In [35]:
myabs(1 + 1im)

LoadError: MethodError: no method matching myabs(::Complex{Int64})
[0mClosest candidates are:
[0m  myabs([91m::Float64[39m) at In[25]:1
[0m  myabs([91m::ComplexF64[39m) at In[29]:1

In [36]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

myabs (generic function with 4 methods)

In [37]:
myabs(-3)

3

# Multiple Dispatch

Julia's dispatch mechanism always chooses the *most specific method* for the given input types.

In [38]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [39]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[38]:5
[2] f(a::Number, b::Number) in Main at In[38]:2
[3] f(a::Number, b) in Main at In[38]:3
[4] f(a, b::Number) in Main at In[38]:4
[5] f(a, b) in Main at In[38]:1

In [40]:
f(1.5, 2)

"a and b are both numbers"

In [41]:
f(1, "Cologne!")

"a is a number"

In [42]:
f(1, 2)

"a and b are both integers"

In [43]:
f("Hello", "World!")

"fallback"

In [44]:
@which f(1, 2)

f(a::Integer, b::Integer) in Main at In[38]:5

In [45]:
@which f(1, "Cologne!")

f(a::Number, b) in Main at In[38]:3

In [46]:
methods(+)

# 208 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:529
[3] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:535
[4] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:397
[5] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:405
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:413
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /opt/julia-1.7.3/share/julia/stdlib/v1.7/Dates/src/periods.jl:372
[8] +(F::LinearAlgebra.Hessenberg, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/hessenberg.jl:640
[9] +(x::Dates.Instant) in Dates at /opt/julia-1.7.3/share/julia/stdlib/v1.7/Dates/src/arithmetic.jl:4
[10] +(A::SparseArrays.AbstractSparseMatrixCSC, B::SparseArrays.AbstractSparseMatrixCSC) in SparseArrays at /opt/julia-1.7.3/share/julia/stdlib/v1.7/SparseArrays/src/sparsematrix.jl:1771
[11] +(A::SparseArrays.AbstractSparseMatrixCSC, B::Array) in SparseArrays at /opt/julia-1.7.3/share/julia/stdlib/v1.7/SparseArrays/src/sparsematrix.jl:1774
[12] +(A::SparseArrays.AbstractSparseMatrixCSC, J::LinearAlgebra.UniformScaling) in SparseArrays at /opt/julia-1.7.3/share/julia/stdlib/v1.7/SparseArrays/src/sparsematrix.jl:3754
[13] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.LowerTriangular) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:641
[14] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.UnitLowerTriangular) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:643
[15] +(UL::LinearAlgebra.UnitLowerTriangular, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:182
[16] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:86
[17] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.UpperTriangular) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:636
[18] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.UnitUpperTriangular) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:638
[19] +(x::LinearAlgebra.UpperTriangular, H::LinearAlgebra.UpperHessenberg) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/hessenberg.jl:107
[20] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:86
[21] +(z::Complex, w::Complex) in Base at complex.jl:288
[22] +(A::BitArray, B::BitArray) in Base at bitarray.jl:1180
[23] +(r1::StepRangeLen{T, R}, r2::StepRangeLen{T, R}) where {R<:Base.TwicePrecision, T} in Base at twiceprecision.jl:613
[24] +(r1::StepRangeLen{T, S}, r2::StepRangeLen{T, S}) where {T, S} in Base at range.jl:1370
[25] +(r1::LinRange{T}, r2::LinRange{T}) where T in Base at range.jl:1354
[26] +(x::T, y::Integer) where T<:AbstractChar in Base at char.jl:237
[27] +(index1::CartesianIndex{N}, index2::CartesianIndex{N}) where N in Base.IteratorsMD at multidimensional.jl:114
[28] +(J::LinearAlgebra.UniformScaling) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:150
[29] +(J::LinearAlgebra.UniformScaling, x::Number) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:145
[30] +(J1::LinearAlgebra.UniformScaling, J2::LinearAlgebra.UniformScaling) in LinearAlgebra at /opt/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:151
[31] +(J::LinearAlgebra.UniformScaling, B::

In [47]:
@which true + false

+(x::Bool, y::Bool) in Base at bool.jl:162

In [48]:
@which "Hello"*"World!"

*(s1::Union{AbstractChar, AbstractString}, ss::Union{AbstractChar, AbstractString}...) in Base at strings/basic.jl:260

Julia's standard functions are not special by any means.

We can easily modify or add methods to them as well.

In [49]:
import Base: + # we have to import functions to override/extend them
+(x::String, y::String) = x * " " * y

+ (generic function with 209 methods)

In [50]:
"Hello" + "Cologne!"

"Hello Cologne!"

(**Important note**: as we neither own the `+` function nor the `String` type the above is **type piracy** and should generally be avoided.)

Any function based on the `+` operation can now handle `String`s as well.

In [51]:
sum(["This", "works", "although", "we", "never", "touched", "sum!"])

"This works although we never touched sum!"

It happens rarely, but it can happen that there is no unique most specific method:

In [52]:
f(x::Int, y::Any) = println("int")
f(x::Any, y::String) = println("string")
f(3, "test")

LoadError: MethodError: f(::Int64, ::String) is ambiguous. Candidates:
  f(x::Int64, y) in Main at In[52]:1
  f(a::Number, b) in Main at In[38]:3
  f(x, y::String) in Main at In[52]:2
Possible fix, define
  f(::Int64, ::String)

# Parametric types

Types can have *type parameters*. The most prominent example is Julia's array type.

In [53]:
M = rand(2,2)

2×2 Matrix{Float64}:
 0.376044  0.996651
 0.578592  0.367891

In [54]:
typeof(M)

Matrix{Float64} (alias for Array{Float64, 2})

Here, `Array` is a parametric array datatype. Its type parameters `Float64` and `2` indicate the type of the element the array can hold and and its dimensionality. Hence, we have a matrix of floating point numbers.

This generalizes as expected. Here, a matrix of `String`s:

In [55]:
M = fill("Cologne", 2,2)

2×2 Matrix{String}:
 "Cologne"  "Cologne"
 "Cologne"  "Cologne"

In [56]:
eltype(M)

String

We can also nest parametric types. This is a vector of matrices of `Float64`s.

In [57]:
v = [rand(2,2) for i in 1:3]

3-element Vector{Matrix{Float64}}:
 [0.7548457070337264 0.6691939625121577; 0.30557734416677873 0.49784908012187656]
 [0.465136429307647 0.9277104457442625; 0.3215263827455208 0.6578866095522569]
 [0.10175260492621852 0.9367163619602478; 0.5572210754422865 0.6322802074695789]

In [58]:
eltype(v)

Matrix{Float64} (alias for Array{Float64, 2})

Another example of a parametric type is the `Tuple`.

In [59]:
(1,2.0,"3")

(1, 2.0, "3")

In [60]:
typeof((1,2.0,"3"))

Tuple{Int64, Float64, String}

### Type parameters in function signatures

Naive approach:

In [61]:
myfunc(v::Vector{Real}) = "I'm a real vector!"

myfunc (generic function with 1 method)

In [62]:
myfunc([1.0,2.0,3.0])

LoadError: MethodError: no method matching myfunc(::Vector{Float64})
[0mClosest candidates are:
[0m  myfunc([91m::Vector{Real}[39m) at In[61]:1

Huh? What's going on?

Note although we have

In [63]:
Float64 <: Real

true

parametric types have the following (perhaps somewhat counterintuitive) property

In [64]:
Vector{Float64} <: Vector{Real}

false

In [65]:
[1.0,2.0,3.0] isa Vector{Real}

false

How can we understand the behavior above? The crucial point is that `Vector{Real}` is a **concrete** container type despite the fact that `Real` is an abstract type. Specifically, it describes a **heterogeneous** vector of values that individually can be of any type `T <: Real`.

In [66]:
isconcretetype(Vector{Real})

true

In [67]:
Real[1, 2.2, 13f0]

3-element Vector{Real}:
  1
  2.2
 13.0f0

As we have learned above, concrete types are the leafes of the type tree and **cannot** have any subtypes. Hence it is only consistent to have...

In [68]:
Vector{Float64} <: Vector{Real}

false

What we often actually *mean* when writing `myfunc(v::Vector{Real}) = ...` is

In [69]:
myfunc(v::Vector{T}) where T<:Real = "I'm a real vector!"

myfunc (generic function with 2 methods)

In [70]:
myfunc([1.0,2.0,3.0])

"I'm a real vector!"

It works! But what does it mean exactly? First of all, we see that

In [1]:
Vector{Float64} <: Vector{T} where T<:Real

true

Here, `Vector{T} where T <: Real` describes the **set** of concrete `Vector` types whose elements are of any specific single type `T` that is a subtype of `Real`.

Think of it as representing `{{ Vector{Float64}, Vector{Int64}, Vector{Int32}, Vector{AbstractFloat}, ... }}`.

In [2]:
Vector{Int64} <: Vector{T} where T<:Real

true

In [3]:
Vector{AbstractFloat} <: Vector{T} where T<:Real

true

In [4]:
[1.0,2.0,3.0] isa Vector{T} where T<:Real

true

We can also use the `where` notation to write out our naive `Vector{Real}` from above in a more explicit way:

In [5]:
Vector{Real} === Vector{T where T<:Real}

true

Note that the crucial difference is the position of the `where T<:Real` piece, i.e. whether it is inside or outside of the curly braces.

In [6]:
Vector{T where T<:Real} <: Vector{T} where T<:Real

true

In [7]:
(Vector{T} where T<:Real) <: Vector{T where T<:Real}

false

# "Diagonal" dispatch

In [8]:
d(x::T, y::T) where T = "same type"
d(x, y) = "different types"

d (generic function with 2 methods)

In [9]:
d(3, 4)

"same type"

In [10]:
d(3.0, 1.0)

"same type"

In [11]:
d(1, 4.2)

"different types"

# Duck typing examples

### `UnitRange`

In [12]:
x = 1:30

1:30

In [13]:
typeof(x)

UnitRange{Int64}

In [14]:
typeof(x) <: AbstractArray

true

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [15]:
x[3]

3

In [16]:
size(x)

(30,)

In [17]:
eltype(x)

Int64

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [18]:
fieldnames(typeof(x))

(:start, :stop)

or just by inspecting the source code

In [19]:
@which UnitRange(1, 30)

UnitRange(start::T, stop::T) where T<:Real in Base at range.jl:356

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [20]:
@time collect(1:10000000);

  0.039168 seconds (2 allocations: 76.294 MiB, 18.47% gc time)


But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [21]:
@time 1:10000000;

  0.000001 seconds


Yet, in code they *act* the same way.

# Extra: slurping and splatting

In [26]:
f(x...) = println(x) # slurping

f (generic function with 1 method)

In [27]:
f(3, 1.2, "Cologne")

(3, 1.2, "Cologne")


In [28]:
g(x::Vector) = +(x...) # splat vector into addition operation

g (generic function with 1 method)

In [29]:
g([1,2,3])

6

# Core messages of this Notebook

* **Concrete types** describe data structures, i.e. concrete implementations.
* **Abstract types** define the kind of a thing (What is it? What can I do with it?), i.e. an informal interface. This is also known as **duck-typing**.
* A **function** (the what) can have multiple **methods** (the how).
* **Multiple dispatch**: Julia selects the method to run based on the types of all input arguments and chooses the most specialized one.
* Types can have parameters, i.e. `Vector{Float64}`. We can use the notation `T where T<:SomeSuperType` to address *sets* of types.